<a href="https://colab.research.google.com/github/sargyri/Drop_Lev/blob/master/Machine_learning/Lev_droplet_st_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Lev Droplet 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/r2.0rc/site/en/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Determination of ST by NN using coords (rho,theta, volume, volt)

In [ ]:
# Use seaborn for pairplot
#!pip install seaborn
#!pip install talos
#!pip install -q  --no-deps tensorflow-addons~=0.7

In [ ]:
!pip install lmfit

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import pprint
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow_addons as tfa

print(tf.__version__)


Run the next code box only if google drive is not mounted to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

If it **is** mounted continue from here on

Check the connection to google's **GPU**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**Or** check the connection to google's **TPU**

In [ ]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

### Get the data
First download the dataset!

**Option 1**: Upload files

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
csvfilename=str('TritonX100_conci_0.0100_all.csv')
input = pd.read_csv(io.BytesIO(uploaded[csvfilename]))
#input = pd.read_csv(csvfilename, sep="\t")
# Dataset is now stored in a Pandas Dataframe
input.shape

**Option 2**: Import files from drive

In [ ]:
#From the drive folder that is mounted to collab go to the file that you are interested in, 
#right click and copy the path of the file. Paste it in csvfilename:

fileSDS1=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.01054_all.csv'), sep="\t")
fileSDS2=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.02108_all.csv'), sep='\t')
fileSDS3=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.04273_all.csv'), sep='\t')
fileSDS4=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.08828_all.csv'), sep="\t")
fileSDS5=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.25000_all.csv'), sep="\t")
fileSDS6=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.50000_all.csv'), sep="\t")
fileSDS7=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_0.75000_all.csv'), sep="\t")
fileSDS8=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_1.00000_all.csv'), sep="\t")
fileSDS9=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_1.25000_all.csv'), sep="\t")
fileSDS10=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_1.50000_all.csv'), sep="\t")
fileSDS11=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_2.00000_all.csv'), sep="\t")
fileSDS12=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_2.50000_all.csv'), sep="\t")
fileSDS13=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_3.00000_all.csv'), sep="\t")
fileSDS14=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_3.50000_all.csv'), sep="\t")
fileSDS15=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_4.00000_all.csv'), sep="\t")
fileSDS16=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_4.50000_all.csv'), sep="\t")
fileSDS17=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_5.50000_all.csv'), sep="\t")
fileSDS18=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_6.50000_all.csv'), sep="\t")
fileSDS19=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_7.50000_all.csv'), sep="\t")
fileSDS20=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_8.50000_all.csv'), sep="\t")
fileSDS21=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_24.75900_all.csv'), sep="\t")

fileTrit1=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.00250_all.csv'), sep="\t")
fileTrit2=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.00500_all.csv'), sep="\t")
fileTrit3=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.00750_all.csv'), sep="\t")
fileTrit4=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.01000_all.csv'), sep="\t")
fileTrit5=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.01500_all.csv'), sep="\t")
fileTrit6=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.02000_all.csv'), sep="\t")
fileTrit7=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.02500_all.csv'), sep="\t")
fileTrit8=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.03500_all.csv'), sep="\t")
fileTrit9=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.05000_all.csv'), sep="\t")
fileTrit10=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.07500_all.csv'), sep="\t")
fileTrit11=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.10000_all.csv'), sep="\t")
fileTrit12=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.12500_all.csv'), sep="\t")
fileTrit13=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.15000_all.csv'), sep="\t")
fileTrit14=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.17500_all.csv'), sep="\t")
fileTrit15=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.20000_all.csv'), sep="\t")
fileTrit16=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.25000_all.csv'), sep="\t")
fileTrit17=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.30000_all.csv'), sep="\t")
fileTrit18=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_0.45000_all.csv'), sep="\t")
fileTrit19=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_1.00000_all.csv'), sep="\t")
fileTrit20=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_3.00000_all.csv'), sep="\t")
fileTrit21=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/TritonX100/TritonX_conci_10.00000_all.csv'), sep="\t")

fileCTAB1=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.00100_all.csv'), sep="\t")
fileCTAB2=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.00200_all.csv'), sep="\t")
fileCTAB3=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.00500_all.csv'), sep="\t")
fileCTAB4=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.01000_all.csv'), sep="\t")
fileCTAB5=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.01500_all.csv'), sep="\t")
fileCTAB6=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.05500_all.csv'), sep="\t")
fileCTAB7=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.10000_all.csv'), sep="\t")
fileCTAB8=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.15000_all.csv'), sep="\t")
fileCTAB9=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.20000_all.csv'), sep="\t")
fileCTAB10=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.25000_all.csv'), sep="\t")
fileCTAB11=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.30000_all.csv'), sep="\t")
fileCTAB12=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.40000_all.csv'), sep="\t")
fileCTAB13=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.50000_all.csv'), sep="\t")
fileCTAB14=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.60000_all.csv'), sep="\t")
fileCTAB15=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.70000_all.csv'), sep="\t")
fileCTAB16=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.80000_all.csv'), sep="\t")
fileCTAB17=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.85000_all.csv'), sep="\t")
fileCTAB18=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_0.90000_all.csv'), sep="\t")
fileCTAB19=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_1.00000_all.csv'), sep="\t")
fileCTAB20=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_5.00000_all.csv'), sep="\t")
fileCTAB21=pd.read_csv(str('/content/drive/My Drive/DropLev/Levitator_2/Training/CTAB/CTAB_conci_13.15224_all.csv'), sep="\t")

#input=np.vstack([fileSDS1, fileSDS2, fileSDS3, fileSDS4, fileSDS5, fileSDS6, fileSDS7, fileSDS8, fileSDS9, fileSDS10, fileSDS11, 
                 #fileSDS12, fileSDS13, fileSDS14, fileSDS15, fileSDS16, fileSDS17, fileSDS18, fileSDS19, fileSDS20, fileSDS21,
                # fileTrit1, fileTrit2, fileTrit3, fileTrit4, fileTrit5, fileTrit6, fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11,
                # fileTrit12, fileTrit13, fileTrit14, fileTrit15, fileTrit16, fileTrit17, fileTrit18, fileTrit19, fileTrit20, fileTrit21,
                # fileCTAB1, fileCTAB2, fileCTAB3, fileCTAB4, fileCTAB5, fileCTAB6, fileCTAB7, fileCTAB8, fileCTAB9, fileCTAB10, fileCTAB11,
                # fileCTAB12, fileCTAB13, fileCTAB14, fileCTAB15, fileCTAB16, fileCTAB17, fileCTAB18, fileCTAB19, fileCTAB20, fileCTAB21])
# input=np.vstack([fileSDS5, fileSDS6, fileSDS7, fileSDS8, fileSDS9, fileSDS10, fileSDS13, fileSDS14,
#                  fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11,
#                  fileCTAB9, fileCTAB10, fileCTAB11, fileCTAB12,fileCTAB13, fileCTAB14])
# input.shape

#csvfilename=str('/content/drive/My Drive/DropLev/Levitator_2/Training/SDS/SDS_conci_1.50000_ML_data.csv')

#input = pd.read_csv(csvfilename)

In [ ]:
input=np.vstack([fileSDS1, fileSDS2, fileSDS3, fileSDS4, fileSDS5, fileSDS6, fileSDS7, fileSDS19, fileSDS20, fileSDS21,
                 fileTrit1, fileTrit2, fileTrit3, fileTrit4, fileTrit5, fileTrit17, fileTrit18, fileTrit19, fileTrit20, fileTrit21,
                 fileCTAB1, fileCTAB2, fileCTAB3, fileCTAB4, fileCTAB5, fileCTAB15, fileCTAB16, fileCTAB17, fileCTAB18, fileCTAB19, fileCTAB20, fileCTAB21])

# input=np.vstack([fileSDS8, fileSDS9, fileSDS10, fileSDS11, fileSDS12, fileSDS13, fileSDS14, fileSDS15, fileSDS16, fileSDS17, fileSDS18,
#                  fileTrit6, fileTrit7, fileTrit8, fileTrit9, fileTrit10, fileTrit11, fileTrit12, fileTrit13, fileTrit14, fileTrit15, fileTrit16,
#                  fileCTAB6, fileCTAB7, fileCTAB8, fileCTAB9, fileCTAB10, fileCTAB11, fileCTAB12, fileCTAB13, fileCTAB14])

In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('phi_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
column_names.append('height')
column_names.append('width')  
column_names.append('volume')
column_names.append('R_sph')
column_names.append('volt')
column_names.append('Intensity')
column_names.append('time')
column_names.append('Power')
#column_names.append('Ps')
#column_names.append('Ps_err')
column_names.append('centre_x')
column_names.append('centre_y')
column_names.append('surfactant_conc')
column_names.append('Aspect_Ratio')
column_names.append('st')


#raw_dataset = pd.read_csv(csvfilename, sep="\t")
#raw_dataset.columns=column_names

raw_dataset = pd.DataFrame(data=input,  columns=column_names) 
              
raw_dataset.shape

dataset = raw_dataset.copy()

#for i in range(300):
#  dataset.pop('rho_%d' %i)
for i in range(300):
  dataset.pop('phi_%d' %i)
#for i in range(300):
#  dataset.pop('theta_%d' %i)
dataset.pop("height")
dataset.pop("width")
#dataset.pop("volume")
#dataset.pop("R_sph")
#dataset.pop('volt')
dataset.pop('Intensity')
dataset.pop('time')
dataset.pop('Power')
#dataset.pop('Ps')
#dataset.pop('Ps_err')
dataset.pop("centre_x")
dataset.pop("centre_y")
dataset.pop('surfactant_conc')
dataset.pop("Aspect_Ratio")

dataset.tail()


In [ ]:
#Drop 125 values of theta
for i in range(0,125):
  dataset=dataset.drop(dataset.columns[300+i], axis=1)
  i=i+1


#Drop 125 values of rho:
for i in range(0,125):
  dataset=dataset.drop(dataset.columns[i], axis=1)
  i=i+1

dataset.pop('R_sph')

dataset.tail()

In [ ]:
volt=dataset['volt']

In [ ]:
dataset.pop('volt')

In [ ]:
datset=dataset.drop([0, 1, 2, 3,4 ,5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], axis=0)
datset.shape

In [ ]:
volt2=volt.drop([156630, 156631, 156632, 156633, 156634, 156635, 156636, 156637, 156638, 156639, 156640, 156641, 156642, 156643, 156644, 156645,
                 156646, 156647, 156648, 156649, 156650, 156651, 156652, 156653, 156654, 156655, 156656, 156657, 156658, 156659, 156659], axis=0)
len(volt2)

In [ ]:
datset.pop('volt')
datset.pop('volt2')

In [ ]:
volt2=np.asarray(volt2)

In [ ]:
datset['volt']=volt2
datset.tail()
dataset=datset

In [ ]:
#Check that the drop contour is still well defined!
plt.plot(dataset.iloc[150000, 150:300], dataset.iloc[150000, 0:150])

### Show the Volume and ST distribution of the data

In [ ]:
#linex=np.tile(3, 2)
#liney=[0, 0.3]

plt.figure()
sns.distplot(dataset['volume'])
plt.xlabel('Volume [μL]')
plt.ylabel('Count')

plt.figure()
sns.distplot(dataset['st'])
plt.xlabel('Surface tension [mN/m]')
plt.ylabel('Count')

plt.figure()
sns.distplot(dataset['volt'])
plt.xlabel('Voltage [V]')
plt.ylabel('Count')


### Selectively choose data with (almost) constant volume

In [ ]:
new_input=[]
vol_min=2.99
vol_max=3.01
for i in range(len(dataset)):
  if dataset['volume'].iloc[i] >= vol_min and dataset['volume'].iloc[i] <= vol_max:
    vol_ct_data=dataset.iloc[i, :]
    new_input.append(vol_ct_data)
new_input=np.asarray(new_input)
new_input.shape

In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
column_names.append('volume')
column_names.append('R_sph')
column_names.append('volt')
column_names.append('st')

new_dataset=pd.DataFrame(new_input, columns=column_names)
new_dataset.pop('R_sph')
new_dataset.tail()

In [ ]:
plt.figure()
sns.distplot(new_dataset['volt'])
plt.xlabel('Voltage [V]')
plt.ylabel('Count')

plt.figure()
sns.distplot(new_dataset['st'])
plt.xlabel('Surface tension [mN/m]')
plt.ylabel('Count')

In [ ]:
# #Drop 125 values of theta
# for i in range(0,125):
#   new_dataset=new_dataset.drop(new_dataset.columns[300+i], axis=1)
#   i=i+1


# #Drop 125 values of theta
# for i in range(0,125):
#   new_dataset=new_dataset.drop(new_dataset.columns[i], axis=1)
#   i=i+1

# #Drop voltage
# new_dataset.pop('volt')


In [ ]:
# #Drop volume if it's constant
# new_dataset.pop('volume')

In [ ]:
# #Check that the drop contour is still well defined!
# plt.plot(new_dataset.iloc[25, 150:300], new_dataset.iloc[25, 0:150])

In [ ]:
new_input=np.asarray(dataset)

In [ ]:
from lmfit import Model, fit_report, Parameters

def model_fit(th, Ps):
    """
    Express the model that will be used to fit the experimental data (simple approach).

    Input parameters
    ----------
    dB:     deciBell
    th:     angle theta
    Returns
    -------
    Expression of fitting model.

    """
    rho=-((3/(64*gamma))*R_sph**2*Ps**2*Cg_air*(1+((7/5)*(k_o*R_sph)**2)))*(3*(np.cos(th))**2-1)+R_sph
    return rho
data_Ps=[]
Cg_air=1/101325 #Pa**(-1)
k_o=2*np.pi*40/340 #mm^-1

for i in range(len(new_input)):
  gamma=new_input[i, 603]
  R_sph=new_input[i, 601]
  rho=new_input[i, 0:300]
  gmodel = Model(model_fit, nan_policy='omit')
    
  fit_param = Parameters()
  fit_param.add('Ps', value=700, min=10, max=4000)
  
  result = gmodel.fit(rho, th=new_input[i, 300:600], params=fit_param)         #rho_final, th: x and y to be evaluated, respectively
  Ps=result.params['Ps'].value
  data_Ps.append(Ps)
data_Ps=np.asarray(data_Ps)

plt.figure()
plt.plot(data_Ps)
plt.xlabel('Drop')
plt.ylabel('Acoustic pressure [Pa]')

In [ ]:
plt.figure()
plt.plot(new_input[i, 300:600], rho, 'bo')
plt.plot(new_input[i, 300:600], result.best_fit, 'r-', label='best fit')

In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
column_names.append('volume')
column_names.append('R_sph')
column_names.append('volt')
column_names.append('st')

new_dataset=pd.DataFrame(new_input, columns=column_names)

In [ ]:

#Add the calclulated Ps values to the new_dataset dataframe
new_dataset['Ps']=data_Ps

#Drop the R_sph values since we cannot have them in the prediction
new_dataset.pop('R_sph')
new_dataset.pop('volt')
# new_dataset.pop('volume')
new_dataset.tail()

In [ ]:
dataset=new_dataset
#Drop 125 values of theta
for i in range(0,125):
  dataset=dataset.drop(dataset.columns[300+i], axis=1)
  i=i+1


#Drop 125 values of rho:
for i in range(0,125):
  dataset=dataset.drop(dataset.columns[i], axis=1)
  i=i+1

dataset.tail()

In [ ]:
newer_input=[]
Ps_min=1500
Ps_max=2100
for i in range(len(new_dataset)):
  if new_dataset['Ps'].iloc[i] >= Ps_min and new_dataset['Ps'].iloc[i] <= Ps_max:
    Ps_range=new_dataset.iloc[i, :]
    newer_input.append(Ps_range)
newer_input=np.asarray(newer_input)

newer_input.shape

In [ ]:
plt.figure()
plt.plot(newer_input[:, 601])
plt.ylabel('Acoustic pressure [Pa]')
plt.xlabel('droplet')

In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
# column_names.append('volume')
column_names.append('st')
column_names.append('Ps')

new_df=pd.DataFrame(newer_input, columns=column_names)

#Drop ST:
new_df.pop('st')

In [ ]:
stats_input_df=new_df.describe()
stats_input_df=stats_input_df.transpose()
new_df.describe()

In [ ]:
def norm_test(x):
  return (x - stats_input_df['mean']) / stats_input_df['std']

#import model from drive
filename=str('test_training_all_vary_vol_0.h5')
model = tf.keras.models.load_model('/content/drive/My Drive/DropLev/MachineLearningModels/'+filename)

norm_input_array=norm_test(new_df)
prediction=model.predict(norm_input_array).flatten()
true_values=newer_input[:, 351]

plt.figure()
plt.xlabel('Frame number')
plt.ylabel('Surface tension [ mN/m ]')
plt.ylim(20, 75)
plt.plot(prediction, label='Predicted values')
plt.plot(true_values, label='Real values')
plt.legend()

plt.figure()
plt.scatter(true_values, prediction)
plt.title('Normalized new test data')
plt.xlabel('True Values [surface tension - mN/m]')
plt.ylabel('Predictions [surface tension - mN/m]')
plt.axis('equal')
plt.axis('square')
plt.xlim([10,75])
plt.ylim([10,75])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

error = prediction - true_values
plt.figure()
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [mN/m]")
_ = plt.ylabel("Count")
#plt.ylim(0, 30)
plt.show()


Import it using pandas

In [ ]:
column_names=[]

for i in range(300):
  column_names.append('rho_%d' %i)
for i in range(300):
  column_names.append('phi_%d' %i)
for i in range(300):
  column_names.append('theta_%d' %i)
column_names.append('height')
column_names.append('width')  
column_names.append('volume')
column_names.append('R_sph')
column_names.append('volt')
column_names.append('Intensity')
column_names.append('time')
column_names.append('Power')
#column_names.append('Ps')
#column_names.append('Ps_err')
column_names.append('centre_x')
column_names.append('centre_y')
column_names.append('surfactant_conc')
column_names.append('Aspect_Ratio')
column_names.append('st')


#raw_dataset = pd.read_csv(csvfilename, sep="\t")
#raw_dataset.columns=column_names

raw_dataset = pd.DataFrame(data=input,  columns=column_names) 
              


raw_dataset.shape

dataset = raw_dataset.copy()

#for i in range(300):
#  dataset.pop('rho_%d' %i)
#for i in range(300):
#  dataset.pop('phi_%d' %i)
#for i in range(300):
#  dataset.pop('theta_%d' %i)
dataset.pop("height")
dataset.pop("width")
dataset.pop("volume")
#dataset.pop("R_sph")
#dataset.pop('volt')
dataset.pop('Intensity')
dataset.pop('time')
dataset.pop('Power')
#dataset.pop('Ps')
#dataset.pop('Ps_err')
dataset.pop("centre_x")
dataset.pop("centre_y")
dataset.pop('surfactant_conc')
dataset.pop("Aspect_Ratio")

dataset.tail()

###Adding/Changing feutures

In [ ]:
dataset['R_sph']=dataset['R_sph']**2

for i in range(300):
  dataset['theta_%d' %i]= (np.cos(dataset['theta_%d' %i]))**2

### Split the data into train and test

Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

In [ ]:
dataset.pop('R_sph')
dataset.tail()

In [ ]:
dataset['volt']=dataset['volt']*178
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=1)
test_dataset = dataset.drop(train_dataset.index)


### Inspect the data

Have a quick look at the joint distribution of a few pairs of columns from the training set.

In [ ]:
#sns.pairplot(train_dataset[column_names], diag_kind="kde")
#plt.show()

Also look at the overall statistics:

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("st")

train_stats = train_stats.transpose()
train_stats

### Split features from labels

Separate the target value, or "label", from the features. This label is the value that you will train the model to predict.

In [ ]:
train_labels = train_dataset.pop('st')
test_labels = test_dataset.pop('st')

### Normalize the data

Look again at the `train_stats` block above and note how different the ranges of each feature are.

It is good practice to normalize features that use different scales and ranges. Although the model *might* converge without feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice of units used in the input.

Note: Although we intentionally generate these statistics from only the training dataset, these statistics will also be used to normalize the test dataset. We need to do that to project the test dataset into the same distribution that the model has been trained on.

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
#normed_train_data = train_dataset
#normed_test_data = test_dataset


In [ ]:
#to check!

#normed_test_data = tf.nn.batch_normalization(train_dataset, train_stats['mean'], train_stats['std'], offset=None, scale=None, variance_epsilon=1)
#normed_test_data_old = norm(train_dataset)

This normalized data is what we will use to train the model.

Caution: The statistics used to normalize the inputs here (mean and standard deviation) need to be applied to any other data that is fed to the model, along with the one-hot encoding that we did earlier.  That includes the test set as well as live data when the model is used in production.

## The model

### Build the model

Let's build our model. Here, we'll use a `Sequential` model with two densely connected hidden layers, and an output layer that returns a single, continuous value. The model building steps are wrapped in a function, `build_model`, since we'll create a second model, later on.

In [ ]:
STEPS_PER_EPOCH=len(train_dataset)/500
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(200, activation=tf.nn.swish, input_shape=[len(train_dataset.keys())]),
    layers.Dense(150, activation=tf.nn.swish),
    layers.Dense(150, activation=tf.nn.swish),
    layers.Dense(125, activation=tf.nn.swish),
    layers.Dense(125, activation=tf.nn.swish),
    layers.Dense(100, activation=tf.nn.swish),
    # layers.Dense(100, activation=tf.nn.swish),
    layers.Dense(75, activation=tf.nn.swish),
    layers.Dense(50, activation=tf.nn.swish),
    layers.Dense(25, activation=tf.nn.swish),
    layers.Dense(5, activation=tf.nn.swish),
    layers.Dense(1)
  ])

 # optimizer = tf.keras.optimizers.RMSprop(0.0001)

  model.compile(loss='mean_squared_error',
                optimizer=get_optimizer(),
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [ ]:
    # layers.Dense(70, activation=tf.nn.swish),
    # layers.Dense(60, activation=tf.nn.swish),
    # layers.Dense(50, activation=tf.nn.swish),
    # layers.Dense(40, activation=tf.nn.swish),
    # layers.Dense(30, activation=tf.nn.swish),
    # layers.Dense(20, activation=tf.nn.swish),
    # layers.Dense(10, activation=tf.nn.swish),

In [ ]:
model = build_model()

## If there is a model saved on google drive execute the next block

In [ ]:
from tensorflow.keras.models import load_model

# del model 
folder=str('theory_experiment_mix/')
model_name=str('All_surf_all_vary_175points_trial_1.h5')
model = load_model(str("/content/drive/My Drive/DropLev/MachineLearningModels/"+folder+model_name))


### Inspect the model

Use the `.summary` method to print a simple description of the model

In [ ]:
model.summary()


Now try out the model. Take a batch of `10` examples from the training data and call `model.predict` on it.

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

It seems to be working, and it produces a result of the expected shape and type.

### Train the model

Train the model for 1000 epochs, and record the training and validation accuracy in the `history` object.

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 55 == 0: print('')
    print('.', end='')


EPOCHS = 1000


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0, patience=75, verbose=0, mode='auto')


history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[early_stop, PrintDot()])  # callbacks=callbacks_list

Visualize the model's training progress using the stats stored in the `history` object.

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Let's automatically stop training when the validation score doesn't improve. We'll use an *EarlyStopping callback* that tests a training condition for  every epoch. If a set amount of epochs elapses without showing improvement, then automatically stop the training.

You can learn more about this callback [here](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [ST]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$ST^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,2])
  plt.legend()
  plt.show()


plot_history(history)

Let's see how well the model generalizes by using the **test** set, which we did not use when training the model.  This tells us how well we can expect the model to predict when we use it in the real world.

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} mN/m".format(mae))
# keras.metrics.accuracy(normed_test_data, test_labels)

### Make predictions

Finally, predict volume values using data in the testing set:

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.title('Normalized test data')
plt.xlabel('True Values [surface tension - mN/m]')
plt.ylabel('Predictions [surface tension - mN/m]')
plt.axis('equal')
plt.axis('square')
plt.xlim([20,75])
plt.ylim([20,75])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

test_predictions2 = model.predict(normed_train_data).flatten()

plt.scatter(train_labels, test_predictions2)
plt.title('Normalized training data')
plt.xlabel('True Values [surface tension - mN/m]')
plt.ylabel('Predictions [surface tension - mN/m]')
plt.axis('equal')
plt.axis('square')
plt.xlim([20,75])
plt.ylim([20,75])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [surface tension - mN/m]")
_ = plt.ylabel("Count")
# plt.xlim(-20, 20)
plt.show()

# Saving the final Model

When you are satisfied with the model and the accuracy continue from here:

Saving the model in a **SavedModel** format

In [ ]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

In [ ]:
# Save the entire model as a SavedModel.
folder=str('theory_experiment_mix/')
filename=str('All_surf_all_vary_175points_trial_1.h5')
!mkdir -p '/content/drive/My Drive/DropLev/MachineLearningModels' #Ceates a folder named "MachineLearningModels"
model.save('/content/drive/My Drive/DropLev/MachineLearningModels/'+folder+filename)  #Saves the file

The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

In [ ]:
# DropLev_st_prediction_model directory
#!ls content/drive/My Drive/DropLev/MachineLearningModels

# Contains an assets folder, saved_model.pb, and variables folder.
#!ls content/drive/My Drive/DropLev/MachineLearningModels/SDS_1.h5

Saving the model in a **HDF5** format

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
#model.save('content/drive/My Drive/DropLev/MachineLearningModels/'+filename) 

# Reload a fresh Keras model from the saved model:

For **SavedModel** format

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/My Drive/DropLev/MachineLearningModels/'+filename)

# Check its architecture
new_model.summary()

For **HDF5** format

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('DropLev_st_prediction_model.h5')

# Show the model architecture
new_model.summary()

Evaluate the restored model (for both formats)

In [ ]:
# Evaluation
loss, mae, mse = new_model.evaluate(normed_test_data, test_labels, verbose=0)
print('Restored model, Mean absolute error: {:5.2f}'.format(mae))

print(new_model.predict(normed_test_data).shape)